### 02. CNN으로 MNIST 분류하기

#### 1. 모델 이해하기
- 모델의 아키텍처 먼저 공부
- 합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위가 조금 다르다. 
    - 첫번째 표기 방법
        - 합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 본다.
        - 맥스풀링(nn.MaxPool2d)은 풀링 층으로 별도로 명명한다. 
    - 두번째 표기 방법
        - 합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(MaxPool2d)을 하나의 합성곱 층으로 본다.

##### **[모델의 아키텍처는 총 3개의 층으로 구성된다.]**
1. 1번 레이어 : 합성곱 층(Convolutional layer)
- 합성곱(in_channel = 1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화함수 ReLU + 맥스풀링(kernel_size=2, stride=2)
2. 2번 레이어 : 합성곱 층(Convolutional layer)
- 합성곱(in_channel = 32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화함수 ReLU + 맥스풀링(kernel_size=2, stride=2)
3. 3번 레이어 : 전결합층(Fully-Connected layer)
- 특성맵을 펼친다. 
- batch_size x 7 x 7 x 64 -> batch_size x 3136
- 전결합층(뉴런 10개) + 활성화 함수 Softmax

-----------------------------------------------------------------------------------------------------------

#### 2. 모델 구현하기

**2-1.** 필요한 도구 임포트와 입력의 정의

필요한 도구들을 임포트한다. 

In [2]:
import torch
import torch.nn as nn

임의의 텐서를 만든다. 텐서의 크기는 1 x 1 x 28 x 28

In [35]:
# 배치크기 x 채널 x 높이 x 너비의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


**2-2.** 합성곱층과 풀링 선언하기

첫번째 합성곱 층을 구현 : 1채널 짜리를 입력받아 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1이다. 

In [36]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


두번째 합성곱 층을 구현 : 32채널 짜리를 입력받아 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1이다.

In [37]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


맥스 풀링 구현 : 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정된다. 

In [38]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


**2-3.** 구현체를 연결하여 모델 만들기

우선 입력을 첫번째 합성곱층을 통과시키고 합성곱층을 통과시킨 후의 텐서의 크기 확인

In [39]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


- 출력 결과
    - 32채널, 28너비, 28높이의 텐서가 출력
    - 32가 나온 이유는 conv1의 out_channel로 32를 지정해주었기 때문
    - 28이 나온 이유는 패딩을 1폭으로 하고 3x3 커널을 사용하면 크기가 보존된다. 

이제 맥스풀링을 통과시키고 맥스 풀링을 통과한 후의 텐서 크기 확인

In [40]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


- 출력결과
    - 32채널 수 그대로 출력
    - 높이와 너비는 1/2가 되므로 각각 14

다시 두번째 합성곱층에 통과시키고 통과한 후의 텐서의 크기 확인

In [41]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


- 출력결과
    - 64채널의 14너비 14높이의 텐서 출력
    - 64가 나온 이유는 conv2의 out_channel을 64로 지정해주었기 때문
    - 14너비, 14높이가 나온 이유는 패딩을 1폭으로 하고 3x3 커널을 사용하여 크기가 보존됨

이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기 확인

In [42]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


이 텐서를 펼치는 작업을 한다. 그런데 펼쳐지기 전에 **텐서의 n번째 차원** 을 접근하게 해주는 **.size(n)** 에 대해서 먼저 공부

In [43]:
# 첫번째 차원 몇인지 출력
out.size(0)

1

In [44]:
# 두번째 차원 몇인지 출력
out.size(1)

64

In [45]:
# 세번째 차원 몇인지 출력
out.size(2)

7

In [46]:
# 네번째 차원 몇인지 출력
out.size(3)

7

이를 가지고 **.view()** 를 사용하여 텐서를 펼치는 작업

In [31]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


- 출력 결과
    - 배치 차원을 제외하고 모두 하나의 차원으로 통합되었다.

이에 대해서 전결합층(Fully-Connected layer)를 통과

In [32]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


- 출력 결과
    - 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환한다. 

------------------------------------------------------------------------------------------------------------

#### 3. CNN으로 MNIST 분류하기 

필요한 도구들을 임포트

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu가 된다. 

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)

학습에 사용할 파라미터 설정

In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

데이터로더를 사용하여 데이터를 다루기 위해서 **데이터셋**을 정의

In [5]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',               # 다운로드 경로 지정
                          train = True,                       # True로 지정하면 훈련 데이터로 다운로드
                          transform = transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root= 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download=True)                 

데이터로더를 사용하여 **배치 크기**를 지정해준다. 

In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

클래스로 모델을 설계

In [7]:
class CNN(torch.nn.Module) : 
    def __init__(self) : 
        super(CNN, self).__init__()
        
        # 첫번째층
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 두번째 층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x) : 
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC(배치 사이즈를 고려 해야 하므로 맨 앞은 그래도 냅두고(배치자리), 나머지 곱)
        out = self.fc(out)
        return out

CNN 모델 정의

In [8]:
model = CNN().to(device)

비용함수와 옵티마이저 정의

In [22]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

총 배치의 수 출력

In [25]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


배치 크기를 100으로 했으므로 훈련 데이터는 총 60000개란 의미

모델 훈련

In [23]:
for epoch in range(training_epochs) : 
    avg_cost = 0
    
    # 미니 배치 단위로 꺼내온다. (X는 미니배치, Y는 레이블)
    for X, Y in data_loader : 
        # 이미지는 28x28 사이즈, no reshape
        # 레이블은 원-핫 인코딩됨
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

[Epoch:    1] cost = 0.225158006
[Epoch:    2] cost = 0.0619881377
[Epoch:    3] cost = 0.0446749516
[Epoch:    4] cost = 0.0363330655
[Epoch:    5] cost = 0.029910991
[Epoch:    6] cost = 0.0251833797
[Epoch:    7] cost = 0.0205985568
[Epoch:    8] cost = 0.018061744
[Epoch:    9] cost = 0.0143755153
[Epoch:   10] cost = 0.0114146387
[Epoch:   11] cost = 0.0105878217
[Epoch:   12] cost = 0.0102650467
[Epoch:   13] cost = 0.00796501897
[Epoch:   14] cost = 0.00606820127
[Epoch:   15] cost = 0.00661244895


테스트

In [26]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad() : 
    # view()를 안하면 torch.Size([10000, 28, 28])
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

C:\Users\ATIV\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


Accuracy:  0.9871000051498413


In [33]:
X_test.shape

torch.Size([10000, 1, 28, 28])

In [28]:
print(prediction.shape)

torch.Size([10000, 10])


In [31]:
torch.argmax(prediction, 1)

tensor([7, 2, 1,  ..., 4, 5, 6])

In [29]:
correct_prediction

tensor([True, True, True,  ..., True, True, True])